In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
class ASR(nn.Module):
    def __init__(self,input,hidden,output):
        super(ASR,self).__init__()
        self.lstm=nn.LSTM(input_size=input,hidden_size=hidden,batch_first=True,bidirectional=True)
        self.fc=nn.Linear(hidden*2,output)

    def forward(self,x):
        x,(hidden,cell)=self.lstm(x)
        print(x.shape)
        x=self.fc(x[:,-1,:])
        print(x.shape)
        return x



In [59]:
input=128
hidden=1024
output=10

device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
model=ASR(input,hidden,output).to(device)
criter=nn.CrossEntropyLoss()
opt=optim.SGD(model.parameters(),lr=1e-3)


In [60]:

def dummy(batch, input, seq_len, out):
    x = torch.rand(batch, seq_len, input).to(device)
    y = torch.randint(0, out, (batch,)).to(device)
    return x, y


In [61]:
batch = 4
seq_len = 100
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(10):
        inputs, labels = dummy(batch, input, seq_len, output)
        outputs = model(inputs)
        loss = criter(outputs, labels)
        total_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/10:.4f}')

torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
Epoch [1/100], Loss: 2.2966
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
Epoch [2/100], Loss: 2.3030
torc

In [62]:
def get_test_loader(batch_size, input_size, seq_len, output_size, num_samples=40):
    """테스트 데이터 로더를 가상으로 생성합니다. 실제 테스트 데이터셋 대신 사용됩니다."""
    test_data = [dummy(batch_size, input_size, seq_len, output_size) for _ in range(num_samples // batch_size)]
    # 배치의 총 수를 반환합니다.
    num_batches = len(test_data)
    return iter(test_data), num_batches

# 테스트 로더 생성 및 배치의 총 수 받기
test_loader, num_batches = get_test_loader(batch, input, seq_len, output, num_samples=40)

def test(model, criterion, test_loader, num_batches, device):
    model.eval()  # 모델을 평가 모드로 설정
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():  # 그래디언트 계산을 비활성화
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / num_batches
    accuracy = 100 * correct / total
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# 테스트 실행
test(model, criter, test_loader, num_batches, device)


torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
torch.Size([4, 100, 2048])
torch.Size([4, 10])
Test Loss: 2.3021, Accuracy: 5.00%


In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio

class ASR(nn.Module):
    def __init__(self, input, hidden, output):
        super(ASR, self).__init__()
        self.lstm = nn.LSTM(input_size=input, hidden_size=hidden, batch_first=True, bidirectional=True)
        # 양방향 LSTM을 위해 hidden_size * 2를 사용
        self.fc = nn.Linear(hidden * 2, output)

    def forward(self, x):
        x, (hidden, cell) = self.lstm(x)
        # 양방향의 마지막 출력을 결합하기 위해 수정 없음
        x = self.fc(x[:, -1, :])
        return x

input = 128
hidden = 256
output = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASR(input, hidden, output).to(device)
criter = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=1e-3)

def dummy(batch, input, seq_len, out):
    x = torch.rand(batch, seq_len, input).to(device)
    y = torch.randint(0, out, (batch,)).to(device)
    return x, y

batch = 4
seq_len = 100
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(10):
        inputs, labels = dummy(batch, input, seq_len, output)
        outputs = model(inputs)
        loss = criter(outputs, labels)
        total_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/10:.4f}')


Epoch [1/10], Loss: 2.2995
Epoch [2/10], Loss: 2.3065
Epoch [3/10], Loss: 2.3031
Epoch [4/10], Loss: 2.2968
Epoch [5/10], Loss: 2.3011
Epoch [6/10], Loss: 2.3160
Epoch [7/10], Loss: 2.2916
Epoch [8/10], Loss: 2.3043
Epoch [9/10], Loss: 2.3079
Epoch [10/10], Loss: 2.3061


In [91]:
def get_test_loader(batch_size, input_size, seq_len, output_size, num_samples=40):
    """테스트 데이터 로더를 가상으로 생성합니다. 실제 테스트 데이터셋 대신 사용됩니다."""
    test_data = [dummy(batch_size, input_size, seq_len, output_size) for _ in range(num_samples // batch_size)]
    # 배치의 총 수를 반환합니다.
    num_batches = len(test_data)
    return iter(test_data), num_batches

# 테스트 로더 생성 및 배치의 총 수 받기
test_loader, num_batches = get_test_loader(batch, input, seq_len, output, num_samples=40)

def test(model, criterion, test_loader, num_batches, device):
    model.eval()  # 모델을 평가 모드로 설정
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():  # 그래디언트 계산을 비활성화
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / num_batches
    accuracy = 100 * correct / total
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# 테스트 실행
test(model, criter, test_loader, num_batches, device)


AttributeError: 'Tensor' object has no attribute 'lstm'

In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torchaudio.transforms import MelSpectrogram
from torch.utils.data import DataLoader
from tqdm import tqdm


train_data=torchaudio.datasets.LIBRISPEECH(root='data', url='train-clean-100',download=True)
test_data=torchaudio.datasets.LIBRISPEECH(root='data',url='test-clean',download=True)


transform=MelSpectrogram()

vocab=set()
for _,_,utterance,_,_,_ in train_data:
    words=utterance.split()
    vocab.update(words)

word_to_index={word:i for i,word in enumerate(vocab)}
index_to_word={i:word for i,word in word_to_index.items()}

def data_prosess(data,transform,word_to_index):
    spectrogram=[]
    labels=[]
    for waveform,_,utterance,_,_,_ in data:
        spec=transform(waveform).squeeze(0).transpose(0,1)
        label_instance=[word_to_index[word]for word in utterance.split()]
        labels.append(label_instance)
        spectrogram.append(spec)
    return spectrogram,labels



train_loader = DataLoader(train_data, batch_size=1, shuffle=True,
                          collate_fn=lambda x: data_process(x, transform, word_to_index))
test_loader = DataLoader(test_data, batch_size=1, shuffle=False,
                         collate_fn=lambda x: data_process(x, transform, word_to_index))

In [107]:
class SRM(nn.Module):
    def __init__(self, input, hidden, output):
        super(SRM, self).__init__()
        self.lstm = nn.LSTM(input_size=input, hidden_size=hidden, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden * 2, output)  # 양방향 고려
        self.gelu = nn.GELU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return self.gelu(x)
        

In [108]:
device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
print(device)

input=128
hidden=256
output=len(vocab)

model=SRM(input,hidden,output).to(device)
model
criter=nn.CrossEntropyLoss()
opt=optim.SGD(model.parameters(),lr=1e-3)

cuda


In [109]:
def train(model,num_epochs,train_loader,criter,opt,device):
    model.train()
    for epoch in range(num_epochs):
        train_loss=0
        progress_bar=tqdm(train_loader,total=len(train_loader),desc=f'Epoch {epoch+1}/{num_epochs}')#프로그레스바
        for spechtrogram,labels in progress_bar:
            spechtrogram.to(device)
            labels=torch.tensor(labels).to(device)
            output=model(spechtrogram)
            loss=criter(output,labels)
            opt.zero_grad()
            loss.backward()
            opt.step()
            train_loss+=loss.item()
        avg_loss=train_loss/len(train_loss)
        print(f'epoch:{epoch}/{num_epochs}, loss:{avg_loss}')

num_epochs=10
train(model,num_epochs,train_loader,criter,opt,device)

ValueError: too many values to unpack (expected 5)

In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torchaudio.transforms import MelSpectrogram
from torch.utils.data import DataLoader
from tqdm import tqdm

# 데이터셋 로드
train_data = torchaudio.datasets.LIBRISPEECH(root='data', url='train-clean-100', download=True)
test_data = torchaudio.datasets.LIBRISPEECH(root='data', url='test-clean', download=True)

# MelSpectrogram 변환기 인스턴스화
transform = MelSpectrogram()

# 어휘집 생성
vocab = set()
for _, _, utterance, _, _, _ in train_data:
    words = utterance.split()  # 오타 수정: uttreance -> utterance
    vocab.update(words)

word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in word_to_index.items()}

# 데이터 전처리 함수
def data_process(batch, transform, word_to_index):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []

    for waveform, _, utterance, _, _, _ in batch:
        spec = transform(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        
        label = [word_to_index[word] for word in utterance.split()]
        labels.extend(label)  # 레이블을 하나의 긴 리스트로
        label_lengths.append(len(label))
        
        input_lengths.append(spec.shape[0])  # spectrogram의 길이 추가

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1)  # 배치 차원 추가
    labels = torch.tensor(labels, dtype=torch.int64)
    
    return spectrograms, labels, torch.tensor(input_lengths), torch.tensor(label_lengths)


# DataLoader
train_loader = DataLoader(train_data, batch_size=1, shuffle=True,
                          collate_fn=lambda x: data_process(x, transform, word_to_index))
test_loader = DataLoader(test_data, batch_size=1, shuffle=False,
                         collate_fn=lambda x: data_process(x, transform, word_to_index))

# 모델 정의
class SRM(nn.Module):
    def __init__(self, input, hidden, output):
        super(SRM, self).__init__()
        self.lstm = nn.LSTM(input_size=input, hidden_size=hidden, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden * 2, output)  # 양방향 LSTM 출력 크기 조정
        self.gelu = nn.GELU()

    def forward(self, x):  # 메서드 정의 수정: 인자 순서 변경 및 오타 수정
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # 모든 시간 단계에서 마지막 출력 사용
        return self.gelu(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 인스턴스화 및 하이퍼파라미터 설정
model = SRM(input=128, hidden=256, output=len(vocab)).to(device)
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)  # blank는 어휘집에서 "빈" 토큰을 나타내는 인덱스입니다.
optimizer = optim.SGD(model.parameters(), lr=1e-3)

# 학습 함수
import torch
from torch import nn, optim
from tqdm import tqdm

def train(model, num_epochs, train_loader, criterion, optimizer, device):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')

        for batch_idx, (spectrograms, labels, input_lengths, label_lengths) in enumerate(progress_bar):
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            # 모델을 통해 Forward Propagation 실행
            optimizer.zero_grad()
            output = model(spectrograms)  # output shape: (T, N, C) where T is the input length, N is the batch size, C is number of classes (including blank)

            # output의 로그 확률 계산 및 CTCLoss 계산을 위한 조정
            output_log_softmax = nn.functional.log_softmax(output, dim=2)
            output_lengths = torch.full(size=(output.size(1),), fill_value=output.size(0), dtype=torch.long)  # 모든 배치에 대해 출력 길이를 동일하게 가정

            # CTCLoss 계산
            loss = criterion(output_log_softmax, labels, output_lengths, label_lengths)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'Loss': f'{total_loss/(batch_idx+1):.4f}'})

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch: {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}')


# 학습 실행
num_epochs = 10
train(model, num_epochs, train_loader, criterion, optimizer, device)





Epoch 1/10:   0%|                                     | 0/28539 [00:00<?, ?it/s]


ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead

In [117]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.transforms import MelSpectrogram
from tqdm import tqdm

train_data = torchaudio.datasets.LIBRISPEECH(root='./data', url='train-clean-100', download=True)

vocab = set()
for _, _, utterance, _, _, _ in train_data:
    words = utterance.lower().split()  # 모든 단어를 소문자로 변환
    vocab.update(words)

# 어휘집에 'UNK' 토큰 추가
vocab.add('UNK')

# 단어를 인덱스로 매핑
word_to_index = {word: i + 1 for i, word in enumerate(vocab)}  # 0 인덱스는 CTC 'blank'를 위해 예약

def data_process(batch, transform, word_to_index):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for waveform, _, utterance, _, _, _ in batch:
        spec = transform(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        # 어휘집에 없는 단어를 위한 처리 추가
        label = [word_to_index.get(word.lower(), word_to_index['UNK']) for word in utterance.split()]
        labels.extend(label)
        label_lengths.append(len(label))
        input_lengths.append(spec.shape[0])
    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).transpose(1, 2)
    labels = torch.tensor(labels, dtype=torch.long)
    return spectrograms, labels, torch.tensor(input_lengths, dtype=torch.long), torch.tensor(label_lengths, dtype=torch.long)


class SRM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(SRM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        # CTC Loss를 위해 log_softmax 적용 필요 없음 (PyTorch >= 1.8.0)
        return x
        
def train(model, num_epochs, train_loader, criterion, optimizer, device):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}')
        for batch_idx, (spectrograms, labels, input_lengths, label_lengths) in progress_bar:
            spectrograms, labels = spectrograms.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(spectrograms)
            output = nn.functional.log_softmax(output, dim=2)
            input_lengths = torch.full(size=(spectrograms.size(0),), fill_value=output.size(0), dtype=torch.long, device=device)
            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({'Loss': f'{total_loss/(batch_idx+1):.4f}'})



transform = MelSpectrogram()
train_loader = DataLoader(train_data, batch_size=1, shuffle=True, collate_fn=lambda x: data_process(x, transform, word_to_index))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRM(input_size=128, hidden_size=256, output_size=len(vocab), num_layers=1)
criterion = nn.CTCLoss(blank=0, zero_infinity=True)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
train(model, num_epochs, train_loader, criterion, optimizer, device)



Epoch 1/10:   0%|                                     | 0/28539 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,
Epoch 1/10:   0%|                                     | 0/28539 [00:00<?, ?it/s]


RuntimeError: shape '[1004544, 1]' is invalid for input of size 131072